######spark.ml实现Nabayes

In [1]:
from pyspark.context import SparkContext
from pyspark.sql import Row
from pyspark.sql.session import SparkSession
from pyspark.ml.classification import NaiveBayes
from pyspark.mllib.regression import LabeledPoint

In [2]:
# 使用本地spark
#sc.stop()
sc = SparkContext('local', 'pyspark')
#建立spark会话
spark = SparkSession.builder\
    .master('local')\
    .appName('spark_mllib')\
    .config('spark.sql.warehouse.dir', 'file:///F:/workspace/work/project/spark/spark-warehouse')\
    .getOrCreate()

In [3]:
from pyspark.ml import linalg as ml_linalg
from pyspark.ml.linalg import Vector as MLVector, Vectors as MLVectors
#from pyspark.mllib.linalg import Vector as MLLibVector, Vectors as MLLibVectors
from pyspark.sql.types import DoubleType
#from pyspark.mllib.linalg import Vectors
from pyspark.ml.linalg import Vectors

'''
def as_mllib(v):
    if isinstance(v, ml_linalg.SparseVector):
        return MLLibVectors.sparse(v.size, v.indices, v.values)
    elif isinstance(v, ml_linalg.DenseVector):
        return MLLibVectors.dense(v.toArray())
    else:
        raise TypeError("Unsupported type: {0}".format(type(v)))
'''

# Load and parse the data
def parseline(line):
    values = line.split('\t')
    label = DoubleType(values[-1])
    features = Vectors.dense([float(x) for x in values[:-1]])
    return LabeledPoint(label=label, features=features)

# Load and parse the data
def parseRow(line):
    row = line.split('\t')
    return Row(labelpoint=row[-1],
               features=Vectors.dense([float(x) for x in row[:-1]]))

path = r'F:\workspace\work\project\MLlib\classification'
data = sc.textFile(path+'\\'+'classify_data.txt')
#转成classifier需要的数据格式
parsedData = data.map(parseRow)
dataset = parsedData.toDF()
#from pyspark.mllib.utils import MLUtils
#dataset.featuers = MLUtils.convertVectorColumnsFromML(dataset.featuers, "features")

In [4]:
print type(dataset)
print dataset.show()

<class 'pyspark.sql.dataframe.DataFrame'>
+-----------------+----------+
|         features|labelpoint|
+-----------------+----------+
|[5.1,3.5,1.4,0.2]|         0|
|[4.9,3.0,1.4,0.2]|         0|
|[4.7,3.2,1.3,0.2]|         0|
|[4.6,3.1,1.5,0.2]|         0|
|[5.0,3.6,1.4,0.2]|         0|
|[5.4,3.9,1.7,0.4]|         0|
|[4.6,3.4,1.4,0.3]|         0|
|[5.0,3.4,1.5,0.2]|         0|
|[4.4,2.9,1.4,0.2]|         0|
|[4.9,3.1,1.5,0.1]|         0|
|[5.4,3.7,1.5,0.2]|         0|
|[4.8,3.4,1.6,0.2]|         0|
|[4.8,3.0,1.4,0.1]|         0|
|[4.3,3.0,1.1,0.1]|         0|
|[5.8,4.0,1.2,0.2]|         0|
|[5.7,4.4,1.5,0.4]|         0|
|[5.4,3.9,1.3,0.4]|         0|
|[5.1,3.5,1.4,0.3]|         0|
|[5.7,3.8,1.7,0.3]|         0|
|[5.1,3.8,1.5,0.3]|         0|
+-----------------+----------+
only showing top 20 rows

None


In [5]:
#StringIndexer将一列labels转译成[0,labels基数)的index
from pyspark.ml.feature import StringIndexer

labeled = StringIndexer(inputCol="labelpoint", outputCol="label")
datas = labeled.fit(dataset).transform(dataset)
datas.show()

+-----------------+----------+-----+
|         features|labelpoint|label|
+-----------------+----------+-----+
|[5.1,3.5,1.4,0.2]|         0|  0.0|
|[4.9,3.0,1.4,0.2]|         0|  0.0|
|[4.7,3.2,1.3,0.2]|         0|  0.0|
|[4.6,3.1,1.5,0.2]|         0|  0.0|
|[5.0,3.6,1.4,0.2]|         0|  0.0|
|[5.4,3.9,1.7,0.4]|         0|  0.0|
|[4.6,3.4,1.4,0.3]|         0|  0.0|
|[5.0,3.4,1.5,0.2]|         0|  0.0|
|[4.4,2.9,1.4,0.2]|         0|  0.0|
|[4.9,3.1,1.5,0.1]|         0|  0.0|
|[5.4,3.7,1.5,0.2]|         0|  0.0|
|[4.8,3.4,1.6,0.2]|         0|  0.0|
|[4.8,3.0,1.4,0.1]|         0|  0.0|
|[4.3,3.0,1.1,0.1]|         0|  0.0|
|[5.8,4.0,1.2,0.2]|         0|  0.0|
|[5.7,4.4,1.5,0.4]|         0|  0.0|
|[5.4,3.9,1.3,0.4]|         0|  0.0|
|[5.1,3.5,1.4,0.3]|         0|  0.0|
|[5.7,3.8,1.7,0.3]|         0|  0.0|
|[5.1,3.8,1.5,0.3]|         0|  0.0|
+-----------------+----------+-----+
only showing top 20 rows



In [6]:
datas_1 = datas.select(datas['features'], datas['label'])

In [7]:
print datas_1.show()

+-----------------+-----+
|         features|label|
+-----------------+-----+
|[5.1,3.5,1.4,0.2]|  0.0|
|[4.9,3.0,1.4,0.2]|  0.0|
|[4.7,3.2,1.3,0.2]|  0.0|
|[4.6,3.1,1.5,0.2]|  0.0|
|[5.0,3.6,1.4,0.2]|  0.0|
|[5.4,3.9,1.7,0.4]|  0.0|
|[4.6,3.4,1.4,0.3]|  0.0|
|[5.0,3.4,1.5,0.2]|  0.0|
|[4.4,2.9,1.4,0.2]|  0.0|
|[4.9,3.1,1.5,0.1]|  0.0|
|[5.4,3.7,1.5,0.2]|  0.0|
|[4.8,3.4,1.6,0.2]|  0.0|
|[4.8,3.0,1.4,0.1]|  0.0|
|[4.3,3.0,1.1,0.1]|  0.0|
|[5.8,4.0,1.2,0.2]|  0.0|
|[5.7,4.4,1.5,0.4]|  0.0|
|[5.4,3.9,1.3,0.4]|  0.0|
|[5.1,3.5,1.4,0.3]|  0.0|
|[5.7,3.8,1.7,0.3]|  0.0|
|[5.1,3.8,1.5,0.3]|  0.0|
+-----------------+-----+
only showing top 20 rows

None


In [10]:
# Split the data into train and test
train, test = datas.randomSplit([0.6, 0.4], seed=11)

In [11]:
# create the trainer and set its parameters
nb = NaiveBayes(smoothing=1.0, modelType="multinomial")

# train the model
model = nb.fit(train)

# select example rows to display.
predictions = model.transform(test)
predictions.show()

+-----------------+----------+-----+--------------------+--------------------+----------+
|         features|labelpoint|label|       rawPrediction|         probability|prediction|
+-----------------+----------+-----+--------------------+--------------------+----------+
|[4.4,3.0,1.3,0.2]|         0|  0.0|[-10.378264301029...|[0.75026434052157...|       0.0|
|[4.4,3.2,1.3,0.2]|         0|  0.0|[-10.595284439568...|[0.77046131082683...|       0.0|
|[4.7,3.2,1.3,0.2]|         0|  0.0|[-10.809074490258...|[0.77923452554345...|       0.0|
|[4.7,3.2,1.6,0.2]|         0|  0.0|[-11.387299657078...|[0.74021800624473...|       0.0|
|[4.8,3.0,1.4,0.1]|         0|  0.0|[-10.492079113781...|[0.77266770178644...|       0.0|
|[4.8,3.0,1.4,0.3]|         0|  0.0|[-11.220039734665...|[0.72471766503686...|       0.0|
|[4.8,3.1,1.6,0.2]|         0|  0.0|[-11.350052938039...|[0.73271058344836...|       0.0|
|[5.0,2.0,3.5,1.0]|         1|  1.0|[-16.872904083261...|[0.12501134970361...|       1.0|
|[5.0,2.3,

In [14]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# compute accuracy on the test set
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",
                                            metricName="accuracy")

accuracy = evaluator.evaluate(predictions)
print "Test set accuracy = " + str(accuracy)

Test set accuracy = 1.0


#####参数寻优grid_search

In [ ]:
import pyspark.ml.tuning as tune



###########模型结果展示

In [ ]:
#混淆矩阵